In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error



from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

import optuna
from sklearn.metrics import r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('data_version_4.csv')
df.head()

,Unnamed: 0,House Number,City,Parking Spaces,Bedrooms,Bathrooms,Servant Quarters,Kitchens,Store Rooms,price,Purpose,Location,Age Possession,area,price_per_sqft,colony,province,society,property Type,area_room_ratio
0,3,52061409,Rawalpindi,1.0,3.0,3.0,0.0,1.0,0.0,1.50,For Sale,"Bahria Town Rawalpindi, Rawalpindi, Punjab,",Relatively New,1361.0,11021.0,Bahria Town Phase 8,Punjab,Bahria Town Rawalpindi,Houses,453.666667
1,4,52010487,Rawalpindi,3.0,6.0,6.0,1.0,2.0,1.0,4.75,For Sale,"Bahria Town Rawalpindi, Rawalpindi, Punjab,",New Property,3812.0,12461.0,Bahria Greens,Punjab,Bahria Town Rawalpindi,Houses,635.333333
2,5,42644685,Rawalpindi,2.0,3.0,3.0,1.0,1.0,1.0,2.25,For Sale,"Askari 14, Rawalpindi, Punjab,",Moderately Old,2722.0,8266.0,Askari 14,Punjab,Askari 14,Houses,907.333333
3,6,52099738,Rawalpindi,3.0,2.0,2.0,0.0,1.0,1.0,1.15,For Sale,"Bahria Town Rawalpindi, Rawalpindi, Punjab,",Relatively New,1171.0,9821.0,Bahria Town,Punjab,Bahria Town Rawalpindi,Flats,585.500000
4,7,51956291,Rawalpindi,1.0,4.0,5.0,0.0,0.0,0.0,1.75,For Sale,"Defence Road, Rawalpindi, Punjab,",Undefined,1361.0,12858.0,Defence Road,Punjab,Defence Road,Houses,340.250000


In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.drop(['House Number','society','price_per_sqft','Location','area_room_ratio','Purpose'],axis=1,inplace=True)
df['property Type'].replace({'Upper':"Houses",'Lower':"Houses"},inplace=True)

In [4]:
df['property Type'].value_counts()

,count
property Type,
Houses,689
Flats,70


In [5]:
columns_to_encode =['property Type']

In [6]:
df['Parking Spaces']=df['Parking Spaces'].astype(int)
df['Bedrooms']=df['Bedrooms'].astype(int)
df['Bathrooms']=df['Bathrooms'].astype(int)
df['Servant Quarters']=df['Servant Quarters'].astype(int)
df['Kitchens']=df['Kitchens'].astype(int)
df['Store Rooms']=df['Store Rooms'].astype(int)

In [7]:
cat_cols = ["City", "Age Possession", "colony", "province", "property Type"]
for col in cat_cols:
    df[col] = df[col].astype("category")

In [8]:
X = df.drop(columns=['price'])
y = df['price']

In [9]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [10]:
!pip install optuna

In [11]:
# import optuna
# import pandas as pd
# import joblib
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.svm import SVR
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
# from sklearn.neural_network import MLPRegressor
# from xgboost import XGBRegressor
# from sklearn.metrics import r2_score, mean_absolute_error
# from sklearn.model_selection import train_test_split

# def reduce_overfitting(model, X_train, y_train, X_test, y_test):
#     """Reduces overfitting by pruning the model and applying regularization."""
#     if isinstance(model, (RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, XGBRegressor)):
#         model.set_params(n_estimators=100, max_depth=10)
#     if isinstance(model, (Ridge, Lasso, SVR, MLPRegressor)):
#         model.set_params(alpha=0.1)
#     model.fit(X_train, y_train)
#     return model

# def train_model(model, X_train, y_train):
#     """Trains the model on the training data."""
#     model.fit(X_train, y_train)
#     return model

# def objective(trial, model, X_train, y_train, X_test, y_test):
#     """Objective function for Optuna hyperparameter tuning."""
#     params = {}
#     if isinstance(model, (RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, XGBRegressor)):
#         params['n_estimators'] = trial.suggest_int('n_estimators', 50, 300)
#         params['max_depth'] = trial.suggest_int('max_depth', 3, 20)
#     if isinstance(model, XGBRegressor):
#         params['learning_rate'] = trial.suggest_float('learning_rate', 0.005, 0.3, log=True)
#         params['subsample'] = trial.suggest_float('subsample', 0.6, 1.0)
#         params['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.6, 1.0)
#         params['gamma'] = trial.suggest_float('gamma', 0, 5)
#     if isinstance(model, AdaBoostRegressor):
#         params['n_estimators'] = trial.suggest_int('n_estimators', 50, 300)
#         params['learning_rate'] = trial.suggest_float('learning_rate', 0.005, 0.3, log=True)

#     model.set_params(**params)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     return mean_absolute_error(y_test, y_pred)

# def find_best_model(X, y_transformed):
#     """Finds the best model and hyperparameters using Optuna, while preventing overfitting."""
#     y_transformed = (y_transformed - y_transformed.min()) / (y_transformed.max() - y_transformed.min())

#     preprocessor = ColumnTransformer(
#         transformers=[
#             ('num', StandardScaler(), ['Parking Spaces', 'Bedrooms', 'Bathrooms', 'Servant Quarters', 'Kitchens', 'Store Rooms', 'area']),
#             ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ['property Type']),
#             ('cat1', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ['Age Possession', 'colony', 'province', 'City'])
#         ],
#         remainder='passthrough'
#     )

#     X_transformed = preprocessor.fit_transform(X)
#     X_train, X_test, y_train, y_test = train_test_split(X_transformed, y_transformed, test_size=0.2, random_state=42)

#     model_dict = {
#         'linear_reg': LinearRegression(),
#         'svr': SVR(),
#         'ridge': Ridge(),
#         'lasso': Lasso(),
#         'decision_tree': DecisionTreeRegressor(),
#         'random_forest': RandomForestRegressor(),
#         'extra_trees': ExtraTreesRegressor(),
#         'gradient_boosting': GradientBoostingRegressor(),
#         'adaboost': AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3)),
#         'mlp': MLPRegressor(),
#         'xgboost': XGBRegressor()
#     }

#     best_model = None
#     best_score = float('inf')
#     best_params = {}

#     for model_name, model in model_dict.items():
#         study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
#         study.optimize(lambda trial: objective(trial, model, X_train, y_train, X_test, y_test), n_trials=50)

#         if study.best_value < best_score:
#             best_score = study.best_value
#             best_model = model.set_params(**study.best_params)
#             best_params = study.best_params

#     print(f"\n🔥 Best Model Selected: {best_model} with MAE: {best_score}")
#     return best_model, best_params, preprocessor

# # print("Training functions are ready to be used in a separate file structure using Cookiecutter.")
# print(best_model)
# print(best_params)


In [12]:
# print(preprocessor)

In [13]:
import optuna
import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

def objective(trial, model, X_train, y_train, X_test, y_test):
    """Objective function for Optuna hyperparameter tuning."""
    params = {}

    if isinstance(model, (RandomForestRegressor, GradientBoostingRegressor, XGBRegressor)):
        params['n_estimators'] = trial.suggest_int('n_estimators', 50, 200)
        params['max_depth'] = trial.suggest_int('max_depth', 3, 15)
    if isinstance(model, XGBRegressor):
        params['learning_rate'] = trial.suggest_float('learning_rate', 0.01, 0.3)

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return mean_absolute_error(y_test, y_pred)

def find_best_model(X, y):
    """Finds the best model using Optuna while limiting tuning time."""
    y = (y - y.min()) / (y.max() - y.min())  # Normalize target variable

    # Data Preprocessing
    preprocessor = ColumnTransformer(
        transformers=[('num', StandardScaler(), ['area']),
                      ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ['property Type'])],
        remainder='passthrough'
    )

    X_transformed = preprocessor.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    model_dict = {
        'ridge': Ridge(),
        'lasso': Lasso(),
        'random_forest': RandomForestRegressor(),
        'gradient_boosting': GradientBoostingRegressor(),
        'xgboost': XGBRegressor()
    }

    best_model = None
    best_score = float('inf')
    best_params = {}

    for model_name, model in model_dict.items():
        print(f"\n🔍 Tuning {model_name}...")

        study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(lambda trial: objective(trial, model, X_train, y_train, X_test, y_test),
                       n_trials=5,  # Reduced trials for speed
                       timeout=120)  # Stops tuning after 2 minutes per model

        if study.best_value < best_score:
            best_score = study.best_value
            best_model = model.set_params(**study.best_params)
            best_params = study.best_params

    print(f"\n🔥 Best Model Selected: {best_model} with MAE: {best_score}")
    return best_model, best_params, preprocessor, X_test, y_test

def evaluate_model(model, X_test, y_test):
    """Evaluates the trained model using R² Score and MAE."""
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    print("\n📊 Model Evaluation Results:")
    print(f"✅ R² Score: {r2:.4f}")
    print(f"✅ MAE: {mae:.4f}")
    return {"r2_score": r2, "mae": mae}

# Find best model and evaluate
best_model, best_params, preprocessor, X_test, y_test = find_best_model(X, y_transformed)
evaluation_results = evaluate_model(best_model, X_test, y_test)

# Save model and preprocessor
joblib.dump(best_model, "best_model.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")

print("\n✅ Model and Preprocessor Saved Successfully!")


[I 2025-03-08 08:13:12,107] A new study created in memory with name: no-name-4d787c43-4a91-4428-891e-c1663eabfcb6
[W 2025-03-08 08:13:12,109] Trial 0 failed with parameters: {} because of the following error: ValueError("could not convert string to float: 'Rawalpindi'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-13-2d205b5be7b2>", line 58, in <lambda>
    study.optimize(lambda trial: objective(trial, model, X_train, y_train, X_test, y_test),
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-13-2d205b5be7b2>", line 23, in objective
    model.fit(X_train, y_train)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


🔍 Tuning ridge...


ValueError: could not convert string to float: 'Rawalpindi'